In [63]:
cd("$(homedir())/Documents/Repos/enso_project.jl")
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\lisah\Documents\Repos\enso_project.jl`


In [64]:
using ReservoirComputing, CSV, DataFrames, DynamicalSystems, Plots, enso_project, Statistics, JLD2

## Perform error analysis on MJO predictions

### prepare data

In [65]:
# load data
header = ["year", "month", "day", "hour", "PC1 coeff", "PC2 coeff", "PC1+PC2 amplitude"]
data = CSV.read("data/mjo_data/mjo_romi.txt", DataFrame; delim=" ", ignorerepeated=true, header=header)
pc1 = data[!,"PC1 coeff"]
pc2 = data[!,"PC2 coeff"];

# perform TDE
D1, τ1, E1 = optimal_separated_de(pc1)
D2, τ2, E2 = optimal_separated_de(pc2);
D1, D2 = Matrix(transpose(Matrix(D1))), Matrix(transpose(Matrix(D2)));

Algorithm stopped due to convergence of E₁-statistic. Valid embedding achieved ✓.
Algorithm stopped due to convergence of E₁-statistic. Valid embedding achieved ✓.


In [66]:
# train, test, val splits
train_size = 10000
val_size = 2000
test_size = size(D1,2) - train_size - val_size

"""train_pc1, train_pc2 = D1[:, 1:train_size],  D2[:, 1:train_size]
val_pc1, val_pc2 = D1[:, train_size+1:train_size+val_size], D2[:, train_size+1:train_size+val_size]
test_pc1, test_pc2 = D1[:, train_size+val_size+1:train_size+val_size+test_size], D2[:, train_size+val_size+1:train_size+val_size+test_size];"""

"train_pc1, train_pc2 = D1[:, 1:train_size],  D2[:, 1:train_size]\nval_pc1, val_pc2 = D1[:, train_size+1:train_size+val_size], D2[:, train_size+1:train_size+val_size]\ntest_pc1, test_pc2 = D1[:, train_size+val_size+1:train_size+val_size+test_size], D2[:, train_size+val_size+1:train_size+val_size+test_size];"

### gather prediction samples

In [72]:
# sample size N
N = 1

# lead time L
L = 60;

### training

In [73]:
spectral_radii = [0.8,0.9,1.0]
sparsities = [0.14,0.1,0.5]
input_scales = [0.1,1.0]
ridge_values = [0.0,1e-5, 1e-6]
reservoir_sizes = [500,700,1000]

param_grid = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values);

In [74]:
function sample_lead_times(L, N, data, train_size, initial_val_size, param_grid)
    datasets_test = zeros(N,L)
    predictions = zeros(N,L)

    for n in 0:(N-1)
        # include data shift n
        train_data = data[:, 1:train_size]
        val_data = data[:, train_size+1:train_size+initial_val_size+n]
        test_data = data[:, train_size+initial_val_size+n+1:train_size+initial_val_size+n+L]

        # train
        esn, W_out, val_loss = enso_project.cross_validate_esn(train_data, val_data, param_grid) 
        prediction = enso_project.esn_eval_pred(esn, W_out, test_data)

        # store 
        datasets_test[n+1,:] = test_data[1,:]
        predictions[n+1,:] = prediction

        # progress control
        println("Finished sample $(n+1)")
    end
    return predictions, datasets_test
end

sample_lead_times (generic function with 1 method)

In [75]:
# pc1
predictions_pc1, datasets_test_pc1 = sample_lead_times(L,N,D1, train_size, val_size,param_grid)


enso_project.ESNHyperparams(500, 0.8, 0.14, 0.1, 0.0)
Validation loss = 15044.143070206772
enso_project.ESNHyperparams(500, 0.8, 0.5, 0.1, 0.0)
Validation loss = 14028.332117730697
Finished sample 1


([-0.27566749046206046 -0.4180764487501046 … 1.3414115528832302 1.0233338669390457], [-0.26594 -0.4044 … 1.12644 0.92548])

In [77]:

CSV.write("C:/Users/lisah/Documents/Repos/enso_project.jl/temp_saves/error_saves/mjo_L$(L)_N$(N)_predictions_pc1.csv", DataFrame(predictions_pc1, :auto))
CSV.write("C:/Users/lisah/Documents/Repos/enso_project.jl/temp_saves/error_saves/mjo_L$(L)_N$(N)_testsets_pc1.csv", DataFrame(datasets_test_pc1, :auto))

"C:/Users/lisah/Documents/Repos/enso_project.jl/temp_saves/error_saves/mjo_L60_N1_testsets_pc1.csv"

In [ ]:
# pc2
predictions_pc2, datasets_test_pc2 = sample_lead_times(L,N,D2, train_size, val_size,param_grid)

In [ ]:

CSV.write("C:/Users/lisah/Documents/Repos/enso_project.jl/temp_saves/error_saves/mjo_L$(L)_N$(N)_predictions_pc2.csv", DataFrame(predictions_pc2, :auto))
CSV.write("C:/Users/lisah/Documents/Repos/enso_project.jl/temp_saves/error_saves/mjo_L$(L)_N$(N)_testsets_pc2.csv", DataFrame(datasets_test_pc2, :auto))